In [1]:
import json
from PIL import Image
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets, utils, models
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
import pandas as pd
from scipy.spatial import distance_matrix
from skbio.stats.distance import permanova
from skbio import DistanceMatrix


In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(pretrained=False)

model.classifier = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.last_channel, 3),
)

model = model.to(device)

# load model weights
weights_path = "./mobilenet-Melanopopillia-800.pth"
assert os.path.exists(weights_path), f"file: '{weights_path}' dose not exist."

#model.load_state_dict(torch.load(weights_path, map_location=device))
checkpoint = torch.load(weights_path, map_location=device)

best_acc = checkpoint['best_acc']

model.load_state_dict(checkpoint['state_dict'])

data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# read class_indict
json_path = './class_indices.json'
assert os.path.exists(json_path), f"file: '{json_path}' dose not exist."
json_file = open(json_path, "r", encoding='utf-8')

class_indict = json.load(json_file)

<All keys matched successfully>

In [ ]:
def img_to_prob(group_root_path, save_path="./csv/", classes=3):
    # Initialize empty dataframes for storing results
    prob_one_group = pd.DataFrame()
    result_copy = pd.DataFrame()

    # Get a list of subdirectories in group_root_path
    group_list = os.listdir(group_root_path)

    # Loop through each group and its subdirectories
    for group in group_list:
        group_path = os.path.join(group_root_path, group)
        for sub_dir in os.listdir(group_path):
            # Get the full path of the subdirectory
            file_path = os.path.join(group_path, sub_dir)

            # Get a list of image paths in the subdirectory
            img_path_list = [
                os.path.join(file_path, i) for i in os.listdir(file_path)
                # Only include files ending in ".jpg" that exist
                if i.endswith(".jpg")
                and os.path.exists(os.path.join(file_path, i))
            ]

            # Skip the subdirectory if there are no image files
            if not img_path_list:
                continue

            # Set the model to evaluation mode
            model.eval()

            # Initialize empty lists for storing results
            result = []
            probability = []

            # Loop through each class and each image in img_path_list
            with torch.no_grad():
                for j in range(classes):
                    for img_path in img_path_list:
                        # Open the image file and apply data transformation
                        img = Image.open(img_path)
                        img = data_transform(img)
                        img = torch.unsqueeze(img, dim=0)

                        # Predict the class probabilities
                        output = torch.squeeze(model(img.to(device))).cpu()
                        predict = torch.softmax(output, dim=0)

                        # Store the result as a string and the probability as a float
                        result.append("{},{:.3},{}".format(
                            img_path, predict[j].numpy(),
                            class_indict[str(j)]))
                        probability.append(predict[j].numpy())

            # Store the result and probability dataframes with the subdirectory name as the column name
            result_copy[sub_dir] = pd.DataFrame(result)
            prob_one_group[sub_dir] = pd.DataFrame(probability)

        # Save the result and probability dataframes as CSV files for the current group
        prob_one_group.to_csv(os.path.join(save_path, group + ".csv"))
        result_copy.to_csv(
            os.path.join(save_path, group + "_" + "all" + ".csv"))

        # Reset the dataframes for the next group
        prob_one_group = pd.DataFrame()
        result_copy = pd.DataFrame()

    return

In [10]:
group_root_path="./xxxxxx/"
save_path="./xxxxx"

In [11]:
prob_csv=img_to_prob(group_root_path, save_path="./csv/", classes=3)